In [9]:
import pandas as pd
import numpy as np
import os

In [10]:
# ここを適宜変える
csv_filename = "SAMPLE.csv"

df = pd.read_csv(os.path.join("Labels", csv_filename), index_col=0)

In [11]:
df.head()

,patient_id,filename,age,sex,target
0,xxxxxxxxxx,2019_00364.csv,20.0,女性,0
1,xxxxxxxxxx,2019_00217.csv,20.0,女性,0
2,xxxxxxxxxx,2019_00691.csv,20.0,女性,0
3,xxxxxxxxxx,2019_00204.csv,20.0,女性,0
4,yyyyyyyyyy,2019_00384.csv,70.0,女性,1


In [12]:
def train_valid_test(df):
    df_unique_patient_id = df["patient_id"].value_counts().rename_axis('patient_id').reset_index(name='counts')
    df_rand=pd.concat([df_unique_patient_id, pd.DataFrame( np.random.rand( len( df_unique_patient_id ) ), columns=['rand'] ) ], axis=1 )
    df = pd.merge(df, df_rand, on='patient_id')
    df.loc[df["rand"] > 0.3, 'split'] = 'TRAIN'
    df.loc[(df["rand"] <= 0.3) & (df["rand"] > 0.15), 'split'] = 'VALID'
    df.loc[df["rand"] <= 0.15, 'split'] = 'TEST'
    print(df["split"].value_counts())
    return df

In [13]:
df_split = train_valid_test(df)

TRAIN    27
TEST      6
VALID     3
Name: split, dtype: int64


In [14]:
df_split.head()

,patient_id,filename,age,sex,target,counts,rand,split
0,xxxxxxxxxx,2019_00364.csv,20.0,女性,0,4,0.645298,TRAIN
1,xxxxxxxxxx,2019_00217.csv,20.0,女性,0,4,0.645298,TRAIN
2,xxxxxxxxxx,2019_00691.csv,20.0,女性,0,4,0.645298,TRAIN
3,xxxxxxxxxx,2019_00204.csv,20.0,女性,0,4,0.645298,TRAIN
4,yyyyyyyyyy,2019_00384.csv,70.0,女性,1,3,0.436076,TRAIN


In [15]:
# check duplicate
train_id = df_split[df_split["split"] == "TRAIN"]["patient_id"].to_list()
valid_id = df_split[df_split["split"] == "VALID"]["patient_id"].to_list()
test_id = df_split[df_split["split"] == "TEST"]["patient_id"].to_list()

if len(set(train_id) & set(valid_id)) > 0 or len(set(train_id) & set(test_id)) > 0 or len(set(test_id) & set(valid_id)) > 0:
    print("Data duplicate......")
else:
    print("Filelist saved")
    df_split = df_split[["filename", "patient_id", "age", "sex", "target", "split"]]
    df_split.to_csv(os.path.join('FileLists', 'FileList_{}.csv'.format(csv_filename.replace(".csv", ""))))
    
    

Filelist saved
